In [ ]:
#load modules 
import ee
import geemap
import ipywidgets as widgets
from ipywidgets import HBox, Label
from datetime import datetime
import csv

In [1]:
# Prevent discovery cache Warning
import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

In [2]:
#initialize earth engine
ee.Initialize()

NameError: name 'ee' is not defined

In [ ]:
#define constants
BUTTON_STYLE = {'button_color':'rgba(28,28,28,.99)'}
BUTTON_LAYOUT = widgets.Layout(width='175px')
BASEMAP = 'CartoDB.DarkMatter'
CENTER = [4.260202, 15.243700]
NOW = datetime.now()

In [ ]:
#functions
def download_to_disk(country_code, year, alerts):
    """download the tile to the GEE disk"""
    task_config = {
        'image':alerts,
        'description':construct_filename(country_code, year),
        'scale': 30,
        'region':country.geometry(),
        'maxPixels': 1e10
    }
    
    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    return task

def create_FIPS_list():
    """create the list of the country code in the FIPS norm using the CSV file provided in utils"""
    fips_list = []
    with open('utils/FIPS_country_code.csv', newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            fips_list.append(row[0])
        
    return fips_list

def get_alerts(country_code, year):
    """ get the alerts from the GLAD project"""
    global country
    global alerts
    
    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    all_alerts  = ee.ImageCollection('projects/glad/alert/UpdResult')
    alerts = all_alerts.select('conf' + year[-2:]).mosaic().clip(country);
    
def construct_filename(country_code, year):
    """return the filename associated with the current task"""
    
    filename = 'alerts_' + country_code + '_' + year + '_' + NOW.strftime("%Y-%m-%d_%H:%M:%S")
    
    return filename

In [3]:
#template
country_code_field =  widgets.Dropdown(
    options=create_FIPS_list(),
    placeholder='Country/province',
)

year_field =  widgets.Dropdown(
    options=['2020','2019'],
    value='2019'
)

output = widgets.Output()

button = widgets.Button(
    description="Use these input!"
)
button.button_style = 'info'
button.style = BUTTON_STYLE
button.layout = BUTTON_LAYOUT

NameError: name 'widgets' is not defined

In [4]:
#define the function used to display the process
def on_button_clicked(b):
    
    global country_code
    global year
    
    with output:
        button.disabled=True
        country_code_field.disabled=True
        year_field.disabled=True
        
        country_code = country_code_field.value.upper()
        year = year_field.value
        
        display_map(country_code)
        get_alerts(country_code, year)
        task = download_to_disk(country_code, year, alerts)
        
def add_layer_to_map(aoi_ee):
    mapa.clear_layers()
    mapa.add_basemap(BASEMAP)
    mapa.addLayer(aoi, {'color': 'FF0000'})
    mapa.centerObject(aoi, zoom=3)
        
def display_map(country_code):
    global mapa
    global aoi
    
    mapa = geemap.Map(center=CENTER)
    mapa.clear_layers()
    mapa.clear_controls()
    mapa.add_basemap(BASEMAP)
    mapa.layout.height='200px'
    display(mapa)
    
    aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    add_layer_to_map(aoi)

In [ ]:
#run
ui1 = HBox([Label('contry code: '), country_code_field])
ui2 = HBox([Label('year: '), year_field])
button.on_click(on_button_clicked)
display(HBox([ui1, ui2, button]))
display(output)